# Linear Algebra
---

An extremely attractive feature of reversible logic is that it maps very naturally to the use of vectors to represent boolean values (inputs/outputs), and matrices to represent operations (circuits). With very little extra work, reversible circuits can be turned into simple linear algebra problems where multiplying an input vector by a circuit matrix gives us the corresponding output.

## 1. Single-Bit Systems

The first step to transition from truth tables to linear algebra, is turn our fundamental unit of information (the bit) into a column vector. We do this by assigning to the two possible values the bit can take ($0$ or $1$) the following representations:

$$ 
\vec{0} = \begin{bmatrix} 1 \\ 0 \end{bmatrix}
\qquad \qquad
\vec{1} = \begin{bmatrix} 0 \\ 1 \end{bmatrix}
$$

I know. At first glance, this seems silly. Adding a whole extra variable appears to be rather redundant, but the many reasons for doing this will become obvious as we progress throughout this textbook. In particular, we will soon see that this vector representation will be a very convenient way to express the likelihood of finding a bit to be either 0 or 1 when we don't have full knowledge of how our circuits are behaving in the presence of, for example, noise.

An easy way to remember this representation is to think of the column vector above as the position of a light switch. If a Boolean $0$ represents a light bulb being off, we can associate this with a light switch being flipped up ($1$ in the top space of the vector, $0$ at the bottom). Similarly, a Boolean $1$ would represent the light bulb being on, which happens when the switch if flipped down ($0$ in the top space of the vector, $1$ at the bottom):

<img src="images\01_03_01_light_switches.png" align = "center" width="300"/>

Now, 